Deceleration of electrons:
Bethe Law is used to calculate the average energy loss,

##### The PAP calculation is performed in two steps:
* Calculation of the area of the distribution, equivalent in principle to that of an atomic number correction
* Direct calculation of the generated intensity, on the basis for the distribution phi(rho.z) which is defined by its area and by the parameters of form adapted to the selected mathematical representation
## This file is made to generalize the code to work like the one of benoit v2



In [11]:
%%html
<style type='text/css'>
.CodeMirror{
font-family: 'Lucida Console';}
</style>

In [12]:
import math
import pandas as pd
import numpy as np
import sqlite3
import ZAF as zaf
# plotting
import matplotlib.pyplot as plt
# inputs
elements = ['Fe','Ni']
elements_df = zaf.get_element(elements)
C = {'Fe':0.1, 'Ni':0.9}
Ec = {'Fe': 7.11, 'Ni': 8.332} ### Check if you have to add all the lines and what to put for H
E = 15 # in keV
take_off_angle = 40
#
excited_element = 'Fe'
excited_line = 'K'
excited_line_mac = 'Ka' 
# import the constants
A = elements_df[['element','molar_mass']]
Z = elements_df[['element','atomic_number']]
density = elements_df[['element','density']]
# import MAC
mac = zaf.get_mac(elements, file_mac='Henke 1993_modified.txt')
print(Z[Z['element'] == 'Fe'])
print(mac)
# Extracting the needed MAC from the Henke database
emission = {'element': excited_element, 'line': excited_line_mac}
mac_df = pd.read_csv("Henke 1993_modified.txt")
sample_mac = pd.DataFrame()
for idx, value in enumerate(elements):
    new_row = pd.DataFrame({'abs': [value], 'mac': [mac_df[(mac_df['zAbs'] == value) & (mac_df['zMes'] == emission['element'])][emission['line']].values[0]] })
    print(new_row)
    sample_mac = sample_mac.append(new_row, ignore_index=True)
print(sample_mac)

  element  atomic_number
0      Fe             26
     zAbs zMes   Ka   Kb     La     Lb     Ll  Ma  Mb  Mz
2325   Fe   Fe   68   52   2151  17518   2879   0   0 NaN
2327   Fe   Ni  344  267  12403  11986  15342   0   0 NaN
2509   Ni   Fe   94   71   2983   2858   4003   0   0 NaN
2511   Ni   Ni   58   43   1802  13818   2403   0   0 NaN
  abs  mac
0  Fe   68
  abs  mac
0  Ni   94
  abs  mac
0  Fe   68
1  Ni   94


C:\Users\khalil\AppData\Local\Temp\ipykernel_18900\4076471001.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_mac = sample_mac.append(new_row, ignore_index=True)
C:\Users\khalil\AppData\Local\Temp\ipykernel_18900\4076471001.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample_mac = sample_mac.append(new_row, ignore_index=True)


In [13]:
#parameters: E,C,Z,A # output M, J,V, Ro, dE/dps
lnJ = 0
M = 0
# M - for sample
for element in elements:
    z_element = Z[Z['element'] == element].atomic_number.values
    a_element = A[A['element'] == element].molar_mass.values
    M += C[element] * z_element/a_element
# Ji, J - for sample
for element in elements: 
    z_element = Z[Z['element'] == element].atomic_number.values
    a_element = A[A['element'] == element].molar_mass.values
    Ji = pow(10,-3) * z_element *(10.04 + 8.25 * math.exp(-z_element/11.22)) # in keV
    lnJi = math.log(Ji)
    lnJ += (C[element]*(z_element/a_element)*lnJi)/M
J = math.exp(lnJ) # in keV
V = E/J
# area parameter (number of primary ionizations)
#1 deceleration of electrons
f_V = 0
Dk = [6.6*pow(10,-6), 1.12*pow(10,-5)*(1.35-0.45*pow(J,2)), 2.2*pow(10,-6)/J]
Pk = [0.78, 0.1, -(0.5-0.25*J)]
for i in range(3):
    f_V += Dk[i]*(E/J)*pow(E/J, Pk[i])
# Ro
Dk = [6.6*pow(10,-6), 1.12*pow(10,-5)*(1.35-0.45*pow(J,2)), 2.2*pow(10,-6)/J]
Pk = [0.78, 0.1, -(0.5-0.25*J)]
Ro = 0
for i in range(3):
    Ro += (1/M)*(pow(J,1-Pk[i])*Dk[i]*pow(E,1+Pk[i])/(1+Pk[i]))
dEdps = -(M/J)*[1/f_V] # keV.g-1.cm2
# calculate the deceleration

print("[+] dE/dps: ", dEdps)
print("[+] V: ", V)
print("[+] J: ", J)
print("[+] M: ", M)
print("[+] Ro [g/cm2]: ", Ro)

[+] dE/dps:  [-194.34381608]
[+] V:  50.275346541764314
[+] J:  0.2983569688085457
[+] M:  [0.47590724]
[+] Ro [g/cm2]:  [0.0009488]


In [14]:
# for the mac database
# Ionization cross section
# calculating 1/S
Uo = E/Ec[excited_element]
Vo = V #### IS THIS TRUE???? ####
k = [6.6*pow(10,-6), 1.12*pow(10,-5)*(1.35-0.45*pow(J,2)), 2.2*pow(10,-6)/J]
Pk = [0.78, 0.1, -(0.5-0.25*J)]
m_k=0.86 + 0.12 * math.exp(-pow((Z[Z['element'] == excited_element].atomic_number.values)/5, 2))
m_values= {'K': m_k,
    'L': 0.82,
    'M': 0.78}
m = m_values['K']
T = [1+Pk[0]-m, 1+Pk[1]-m, 1+Pk[2]-m]
inverse_S = 0
for i in range(3):
    inverse_S += (Uo/(Vo*M)) * k[i]* pow(Vo/Uo, Pk[i]) * (T[i]*pow(Uo, T[i])*math.log(Uo)-pow(Uo,T[i])+1)/pow(T[i],2)
print("1/S: ", inverse_S)

1/S:  [3.67882138e-06]


In [15]:
# Backscatter factor R
# After calculating R/S, representing the primary intensity, and Q^A_l(Eo), one determines the area F of the distribution phi(pz):
# F = (R/S)Q

######################## Appendix I
# Z_b_bar
Z_b_bar = 0
for element in elements:
    z_element = Z[Z['element'] == element].atomic_number.values
    Z_b_bar += C[element] * pow(z_element,1/2)
Z_b_bar = pow(Z_b_bar, 2)
# eta bar
eta_bar = 1.75*pow(10,-3)*Z_b_bar + 0.37*(1-math.exp(-0.015*pow(Z_b_bar, 1.3)))
# W_bar
W_bar = 0.595 * eta_bar/3.7 + pow(eta_bar,4.55)
# J(Uo)
J_Uo = 1+Uo*(math.log(Uo) - 1)
# G(Uo)
q = (2 * W_bar - 1) / (1 - W_bar)
G_Uo = (Uo-1-(1-1/pow(Uo, 1+q))/(1+q))/((2+q)*J_Uo)
R = 1 - eta_bar * W_bar*(1-G_Uo)
print("Backscatter factor R:", R)
# F
cross_section = math.log(Uo)/(pow(Uo, m)*Ec[excited_element]*Ec[excited_element])
F = (R*inverse_S) * cross_section
print('Uo:', Uo)
print('R :', R)
print('1/S: ', inverse_S)
print('Q: ',cross_section)
print('F:', F)

Backscatter factor R: [0.99641447]
Uo: 2.109704641350211
R : [0.99641447]
1/S:  [3.67882138e-06]
Q:  0.007771196380630213
F: [2.84863373e-08]


In [16]:
######## phi(rhoz)
r = 2 - 2.3 * eta_bar
# Phio
phi0 = 1 + 3.3 * (1-1/pow(Uo,r))* pow(eta_bar, 1.2)
# Z_bar_n
lnZ_bar_n = 0
for element in elements:
    z_element = Z[Z['element'] == element].atomic_number.values
    lnZ_bar_n += C[element] * math.log(z_element)
Z_bar_n = math.exp(lnZ_bar_n)
# X
X = pd.DataFrame()
X_tot = 0
for idx, value in enumerate(elements):
    # calculating X
    new_row = pd.DataFrame({'element': [value], 'X': mac[(mac['zAbs'] == value) & (mac['zMes'] == excited_element)][excited_line_mac].values[0] * (1/math.sin(math.radians(take_off_angle)))})
    X = X.append(new_row, ignore_index=True)
    # calculating X total
    X_tot = X_tot + C[value] * mac[(mac['zAbs'] == value) & (mac['zMes'] == excited_element)][excited_line_mac].values[0] * (1/math.sin(math.radians(take_off_angle)))

# Phi: parameters pz, phi0, Z_bar_n, M, J) - output: phi, F1, F2, f_X
Eo = E
D1 = 6.6*pow(10,-6)
D2 =1.12*pow(10,-5)*(1.35-0.45*pow(J,2))
D3 = 2.2*pow(10,-6)/J
Dk = [D1,D2,D3]
Pk = [0.78, 0.1, -(0.5-0.25*J)]
Z_bar = 0
for element in elements:
    z_element = Z[Z['element'] == element].atomic_number.values
    Z_bar += C[element] * z_element

# Rx the range of ionization
# Ro is the total trajectory of the electrons between Eo and El
# Q is a function of overvolatge and atomic number
b = 40/Z_bar
Qo = 1 - 0.535 * math.exp(-pow(21/Z_bar_n, 1.2)) - 2.5*pow(10,-4)*pow(Z_bar_n/20, 3.5)
Q = Qo + (1-Qo)*math.exp(-(Uo-1)/b)
# D
h = pow(Z_bar, 0.45)
D = 1+1/pow(Uo,h)
# Ro
Ro = 0
for i in range(3):
    Ro += (1/M) * pow(J, 1-Pk[i]) * Dk[i] * (pow(Eo,1+Pk[i])-pow(Ec[excited_element], 1+Pk[i])/(1+Pk[i]))

# Rx
Rx = Q * D * Ro

# Rm depth of the maximum of the distribution
G1 = 0.11 * 0.41 * math.exp(-pow(Z_bar/12.75, 0.75))
G2 = 1 - math.exp(-pow((Uo - 1), 0.35)/1.19)
G3 = 1 - math.exp(-(Uo-0.5)*pow(Z_bar,0.4)/4)
Rm = G1 * G2 * G3 * Rx
# d
d = (Rx-Rm)*((F-phi0*Rx/3) * ((Rx-Rm)*F - phi0*Rx*(Rm+Rx/3)))
# Rc
Rc = (3/2)
Rc *= (((F - phi0*Rx/3)/phi0) - pow(d, 1/2)/(phi0*(Rx-Rm)))
A1 = phi0 /(Rm*(Rc - Rx * (Rc/Rm - 1)))
B1 = phi0 - A1 * Rm*Rm
A2 = A1 * (Rc - Rm)/(Rc - Rx)
# DO we need to calculate this phi?
# pz = np.linspace(0,1,100)
# if pz >=0 or pz <= Rc:
#     phi = A1 * (pz - Rm) * (pz - Rm) + B1
# if pz > Rc or pz <= Rx:
#     phi = A2 * (pz - Rx) * (pz - Rx)

# USING X TOT IS THAT CORRECT?????
X = X_tot
# Absorption factor 
F1 = (A1/X)*(((Rc-Rm) * (Rx-Rc-2/X)-2/(X*X))*math.exp(-X*Rc) - (Rc-Rm)*Rx + Rm*(Rc-2/X) + 2/(X*X))
F2 = (A2/X)*(((Rx-Rc) * (Rx-Rc-2/X)-2/(X*X))*math.exp(-X*Rc) - (2/(X*X)*math.exp(-X*Rx)))
f_X = (F1+F2)/F

C:\Users\khalil\AppData\Local\Temp\ipykernel_18900\2941240110.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X = X.append(new_row, ignore_index=True)


In [22]:
def CalcI(C, Eo, mac, excited_element, excited_line_mac, excited_line):
    """
         @mac matrix should contain just of the elements present in the sample or standard
    """
    #parameters: E,C,Z,A # output M, J,V, Ro, dE/dps
    lnJ = 0
    M = 0
    # M - for sample
    for element in elements:
        z_element = Z[Z['element'] == element].atomic_number.values
        a_element = A[A['element'] == element].molar_mass.values
        M += C[element] * z_element/a_element
    # Ji, J - for sample
    for element in elements: 
        z_element = Z[Z['element'] == element].atomic_number.values
        a_element = A[A['element'] == element].molar_mass.values
        Ji = pow(10,-3) * z_element *(10.04 + 8.25 * math.exp(-z_element/11.22)) # in keV
        lnJi = math.log(Ji)
        lnJ += (C[element]*(z_element/a_element)*lnJi)/M
    J = math.exp(lnJ) # in keV
    V = Eo/J
    # area parameter (number of primary ionizations)
    #1 deceleration of electrons - For sample
    f_V = 0
    Dk = [6.6*pow(10,-6), 1.12*pow(10,-5)*(1.35-0.45*pow(J,2)), 2.2*pow(10,-6)/J]
    Pk = [0.78, 0.1, -(0.5-0.25*J)]
    for i in range(3):
        f_V += Dk[i]*(Eo/J)*pow(Eo/J, Pk[i])
    # Ro
    Dk = [6.6*pow(10,-6), 1.12*pow(10,-5)*(1.35-0.45*pow(J,2)), 2.2*pow(10,-6)/J]
    Pk = [0.78, 0.1, -(0.5-0.25*J)]
    Ro = 0
    for i in range(3):
        Ro += (1/M)*(pow(J,1-Pk[i])*Dk[i]*pow(Eo,1+Pk[i])/(1+Pk[i]))
    dEdps = -(M/J)*[1/f_V] # keV.g-1.cm2
    # calculate the deceleration

    print("[+] dE/dps: ", dEdps)
    print("[+] V: ", V)
    print("[+] J: ", J)
    print("[+] M: ", M)
    print("[+] Ro [g/cm2]: ", Ro)
    ##############################################################################################################
    # for the mac database
    # Ionization cross section
    # calculating 1/S
    Uo = Eo/Ec[excited_element]
    print('Excited element: ', excited_element)
    print('Ec: ', Ec[excited_element])
    Vo = V #### IS THIS TRUE???? ####
    m_k=0.86 + 0.12 * math.exp(-pow((Z[Z['element'] == excited_element].atomic_number.values)/5, 2))
    m_values= {'K': m_k,
        'L': 0.82,
        'M': 0.78}
    m = m_values[excited_line]
    T = [1+Pk[0]-m, 1+Pk[1]-m, 1+Pk[2]-m]
    inverse_S = 0
    for i in range(3):
        inverse_S += (Uo/(Vo*M)) * Dk[i]* pow(Vo/Uo, Pk[i]) * (T[i]*pow(Uo, T[i])*math.log(Uo)-pow(Uo,T[i])+1)/pow(T[i],2)
    print("1/S: ", inverse_S)
    ##############################################################################################################
    # Backscatter factor R
    # After calculating R/S, representing the primary intensity, and Q^A_l(Eo), one determines the area F of the distribution phi(pz):
    # F = (R/S)Q

    ######################## Appendix I
    # Z_b_bar
    Z_b_bar = 0
    for element in elements:
        z_element = Z[Z['element'] == element].atomic_number.values
        Z_b_bar += C[element] * pow(z_element,1/2)
    Z_b_bar = pow(Z_b_bar, 2)
    # eta bar
    eta_bar = 1.75*pow(10,-3)*Z_b_bar + 0.37*(1-math.exp(-0.015*pow(Z_b_bar, 1.3)))
    # W_bar
    W_bar = 0.595 * eta_bar/3.7 + pow(eta_bar,4.55)
    # J(Uo)
    J_Uo = 1+Uo*(math.log(Uo) - 1)
    # G(Uo)
    q = (2 * W_bar - 1) / (1 - W_bar)
    G_Uo = (Uo-1-(1-1/pow(Uo, 1+q))/(1+q))/((2+q)*J_Uo)
    R = 1 - eta_bar * W_bar*(1-G_Uo)
    print("Backscatter factor R:", R)
    # F
    cross_section = math.log(Uo)/(pow(Uo, m)*Ec[excited_element]*Ec[excited_element])
    print('CROSS SECTION: ', cross_section)
    F = (R*inverse_S) / cross_section # it was multiplication but corrected by benoit to be division 
    print('Uo:', Uo)
    print('R :', R)
    print('1/S: ', inverse_S)
    print('Q: ',cross_section)
    print('F:', F)
    ##############################################################################################################
    ######## phi(rhoz)
    r = 2 - 2.3 * eta_bar
    # Phio
    phi0 = 1 + 3.3 * (1-1/pow(Uo,r))* pow(eta_bar, 1.2)
    # Z_bar_n
    lnZ_bar_n = 0
    for element in elements:
        z_element = Z[Z['element'] == element].atomic_number.values
        lnZ_bar_n += C[element] * math.log(z_element)
    Z_bar_n = math.exp(lnZ_bar_n)
    # X
    X = pd.DataFrame()
    X_tot = 0
    for idx, value in enumerate(elements):
        # calculating X
        new_row = pd.DataFrame({'element': [value], 'X': mac[(mac['zAbs'] == value) & (mac['zMes'] == excited_element)][excited_line_mac].values[0] * (1/math.sin(math.radians(take_off_angle)))})
        X = X.append(new_row, ignore_index=True)
        # calculating X total
        X_tot = X_tot + C[value] * mac[(mac['zAbs'] == value) & (mac['zMes'] == excited_element)][excited_line_mac].values[0] * (1/math.sin(math.radians(take_off_angle)))

    # Phi: parameters pz, phi0, Z_bar_n, M, J) - output: phi, F1, F2, f_X
    D1 = 6.6*pow(10,-6)
    D2 =1.12*pow(10,-5)*(1.35-0.45*pow(J,2))
    D3 = 2.2*pow(10,-6)/J
    Dk = [D1,D2,D3]
    Pk = [0.78, 0.1, -(0.5-0.25*J)]
    Z_bar = 0
    for element in elements:
        z_element = Z[Z['element'] == element].atomic_number.values
        Z_bar += C[element] * z_element

    # Rx the range of ionization
    # Ro is the total trajectory of the electrons between Eo and El
    # Q is a function of overvolatge and atomic number
    b = 40/Z_bar
    Qo = 1 - 0.535 * math.exp(-pow(21/Z_bar_n, 1/2)) - 2.5*pow(10,-4)*pow(Z_bar_n/20, 3.5) #page 60 why 1/2 instead of 1.2!!!!!!!!!!!!!!!!!!!!!!!!!
    Q = Qo + (1-Qo)*math.exp(-(Uo-1)/b)
    # D
    h = pow(Z_bar, 0.45)
    D = 1+1/pow(Uo,h)
    # Ro
    Ro = 0
    for i in range(3):
        Ro += (1/M) * pow(J, 1-Pk[i]) * Dk[i] * (pow(Eo,1+Pk[i])-pow(Ec[excited_element], 1+Pk[i]))/(1+Pk[i])

    # Rx
    Rx = Q * D * Ro

    # Rm depth of the maximum of the distribution
    print("Uo-1", Uo-1)
    #if (Uo - 1) > 1:
    G1 = 0.11 * 0.41 * math.exp(-pow(Z_bar/12.75, 0.75))
    G2 = 1 - math.exp(-pow((Uo - 1), 0.35)/1.19)
    G3 = 1 - math.exp(-(Uo-0.5)*pow(Z_bar,0.4)/4)
    Rm = G1 * G2 * G3 * Rx
    # else:
    #     Rm = 0
    #     print('WARNING: ELEMENT NOT EXCITED')
    # d
    d = (Rx-Rm)*((F-(phi0*Rx)/3) * ((Rx-Rm)*F - phi0*Rx*(Rm+Rx/3)))
     # Rc
    Rc = (3/2)
    Rc *= (((F - phi0*Rx/3)/phi0) - (pow(d, 1/2))/(phi0*(Rx-Rm)))

    A1 = phi0 /(Rm*(Rc - Rx * ((Rc/Rm) - 1)))
    B1 = phi0 - A1 * Rm*Rm
    A2 = A1 * (Rc - Rm)/(Rc - Rx)
    # DO we need to calculate this phi?
    # pz = np.linspace(0,1,100)
    # if pz >=0 or pz <= Rc:
    #     phi = A1 * (pz - Rm) * (pz - Rm) + B1
    # if pz > Rc or pz <= Rx:
    #     phi = A2 * (pz - Rx) * (pz - Rx)

    # USING X TOT IS THAT CORRECT?????
    X = X_tot
    # Absorption factor correction
    # F1 is intensity after absorption from 0 to Rc
    # F2 is intensity after absorption from Rc to Rx
    print('##################################################')
    print('Rc: ', Rc)
    print('Rx: ', Rx)
    print('Rm: ', Rm)
    F1 = (A1/X)*(((Rc-Rm) * (Rx-Rc-2/X)-2/(X*X))*math.exp(-X*Rc) - (Rc-Rm)*Rx + Rm*(Rc-2/X) + 2/(X*X))
    # Benoit equations
    Int1atRc = math.exp(-X*Rc)*(-B1*(X*X) -  A1*(2 + 2 *X*(Rc - Rm) + (X*X)* (Rc - Rm)*(Rc - Rm)))/(pow(X,3))
    Int1at0 = (-B1*(X*X) -  A1*(2 - 2 * X * Rm + (X*X)* (Rm*Rm)))/(pow(X,3))
    Int1 = Int1atRc - Int1at0
    
    F2 = (A2/X)
    F2 *= (((Rx-Rc) * (Rx-Rc-2/X)-2/(X*X))*math.exp(-X*Rc) - (2/(X*X))*math.exp(-X*Rx))
    # Benoit eq
    Int2atRx = -( 2 * A2 * math.exp(-X*Rx) )/(pow(X,3)) #( 2 * A2 .* exp(-Chi.*Rc) )./(Chi.^3);
    Int2atRc = (A2*math.exp(-X*Rc)*(-2-2*X*(Rc - Rx)-(X*X)*(Rc-Rx)*(Rc - Rx)) )/(pow(X,3))
    Int2 = Int2atRx - Int2atRc
    
    print('TEST######################################')
    print("Rx", Rx)
    print("Rc", Rc)
    print("X", X)
    print('ENDTEST ##################################')
    f_X = (F1+F2)/F
    ##############################################################################################################
    print('Z_bar_n: ', Z_bar_n)
    print('Phi0: ', phi0[0])
    print('M: ', M[0])
    print('F1: ', F1[0])
    print('F1 Benoit', Int1)
    print('F2: ', F2[0])
    print('F2 Benoit', Int2)
    print('f_X: ', f_X[0])
    intensity = C[excited_element] * (Int1 + Int2) # C is the concentration of the measured element
    print('Intensity: ', intensity[0])
    print('\n')

### SHOULD MAKE THIS RUN FOR ALL ELEMENTS
elements = ['Fe','Ni']
excited_line_list = ['K','K']
excited_line_mac_list = ['Ka', 'Ka']
elements_df = zaf.get_element(elements)
C = {'Fe':0.1, 'Ni':0.9}
Ec = {'Fe': 7.11, 'Ni': 8.332} ### Check if you have to add all the lines and what to put for H
take_off_angle = 40
Eo = 15 # keV
for idx, element in enumerate(elements):
    excited_element = element
    excited_line = excited_line_list[idx]
    excited_line_mac = excited_line_mac_list[idx]
    CalcI(C, Eo, mac, excited_element, excited_line_mac, excited_line)

KeyboardInterrupt: 

In [18]:
# from matplotlib.ticker import LinearLocator
# from matplotlib import cm
# elements = ['Li','Al']
# # Regression
# nMsEl = len(elements)
# MACkalpha_Ms = mac_df
# fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(12,12))
# ax.set_title('MAC Heike database')
# # Make data.
# X = np.arange(1, 93, 1)
# Y = np.arange(1, 93, 1)
# X, Y = np.meshgrid(X, Y)
# # R = np.sqrt(X**2 + Y**2)
# print(mac_df)
# # print(mac_df[mac_df['zMes'].isin(elements) | mac_df['zAbs'].isin(elements)])
# #macs = mac_df[mac_df['zMes'].isin(elements) | mac_df['zAbs'].isin(elements)]['Ka']
# macs = mac_df
# # print("The sqrt of the size: ",math.sqrt(2*92*len(elements)-pow(len(elements), 2)))
# #Z = macs.to_numpy().reshape(math.sqrt(2*92*len(elements)-pow(len(elements), 2)),math.sqrt(2*92*len(elements)-pow(len(elements), 2)))
# Z = macs["Ka"].values.reshape(92,92)
# print(macs['Ka'])
# # Plot the surface.
# surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
#                        linewidth=0, antialiased=False)

# # Customize the z axis.
# #ax.set_zlim(-1.01, 1.01)
# ax.zaxis.set_major_locator(LinearLocator(10))
# # A StrMethodFormatter is used automatically
# ax.zaxis.set_major_formatter('{x:.02f}')

# # Add a color bar which maps values to colors.
# fig.colorbar(surf, shrink=0.5, aspect=5)

# plt.show()

In [19]:
# calculating IntPAPstd in each standard

elements = ['Fe']
excited_line_list = ['K']
excited_line_mac_list = ['Ka']
elements_df = zaf.get_element(elements)
C = {'Fe':1}
Ec = {'Fe': 7.11} ### Check if you have to add all the lines and what to put for H
take_off_angle = 40
Eo = 15 # keV
for idx, element in enumerate(elements):
    print(idx)
    excited_element = element
    excited_line = excited_line_list[idx]
    excited_line_mac = excited_line_mac_list[idx]
    CalcI(C, Eo, mac, excited_element, excited_line_mac, excited_line)


0
[+] dE/dps:  [-182.90456792]
[+] V:  53.15814754387855
[+] J:  0.28217687585178713
[+] M:  [0.46557436]
[+] Ro [g/cm2]:  [0.00095118]
Excited element:  Fe
Ec:  7.11
1/S:  [3.69432254e-06]
Backscatter factor R: [0.99681473]
CROSS SECTION:  0.007771196380630213
Uo: 2.109704641350211
R : [0.99681473]
1/S:  [3.69432254e-06]
Q:  0.007771196380630213
F: [0.00047387]
Uo-1 1.109704641350211
##################################################
Rc:  [2.2590821e-06]
Rx:  [0.00060796]
Rm:  [2.23680716e-06]
TEST######################################
Rx [0.00060796]
Rc [2.2590821e-06]
X 105.78922022650805
ENDTEST ##################################
Z_bar_n:  26.000000000000004
Phi0:  1.4624146503174782
M:  0.46557435759692006
F1:  4.607601333109402e-06
F1 Benoit [4.60760202e-06]
F2:  -21.399073781038062
F2 Benoit [0.00046173]
f_X:  -45157.87616450113
Intensity:  0.00046633952820585023


C:\Users\khalil\AppData\Local\Temp\ipykernel_18900\1858711675.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X = X.append(new_row, ignore_index=True)


In [31]:
# CalcI with numpy matrices like Benoit

# calculate the intensity for one element
def CalcI(C, Eo, mac, excited_element, excited_line_mac, excited_line):
    """
         @mac matrix should contain just of the elements present in the sample or standard
    """
    #parameters: E,C,Z,A # output M, J,V, Ro, dE/dps
    # convert the C dictionary into a numpy matrix
    C_numpy = np.zeros((92,1))
    for key in C:
        # 1. find the atomic number of that element
        index = Z[Z['element'] == key].atomic_number.values - 1 # because in python we start counting from 0
        np.put(C_numpy, [index], [C[key]])
        # print("The element", key)
        # print("The index", index)
    C = C_numpy
    lnJ = 0
    M = 0
    # M - for sample
    for element in elements:
        z_element = Z[Z['element'] == element].atomic_number.values
        a_element = A[A['element'] == element].molar_mass.values
        M += C[z_element - 1] * z_element/a_element
    # Ji, J - for sample
    for element in elements: 
        z_element = Z[Z['element'] == element].atomic_number.values
        a_element = A[A['element'] == element].molar_mass.values
        Ji = pow(10,-3) * z_element *(10.04 + 8.25 * math.exp(-z_element/11.22)) # in keV
        lnJi = math.log(Ji)
        lnJ += (C[z_element - 1]*(z_element/a_element)*lnJi)/M
    J = math.exp(lnJ) # in keV
    V = Eo/J
    # area parameter (number of primary ionizations)
    #1 deceleration of electrons - For sample
    f_V = 0
    Dk = [6.6*pow(10,-6), 1.12*pow(10,-5)*(1.35-0.45*pow(J,2)), 2.2*pow(10,-6)/J]
    Pk = [0.78, 0.1, -(0.5-0.25*J)]
    for i in range(3):
        f_V += Dk[i]*(Eo/J)*pow(Eo/J, Pk[i])
    # Ro
    Dk = [6.6*pow(10,-6), 1.12*pow(10,-5)*(1.35-0.45*pow(J,2)), 2.2*pow(10,-6)/J]
    Pk = [0.78, 0.1, -(0.5-0.25*J)]
    Ro = 0
    for i in range(3):
        Ro += (1/M)*(pow(J,1-Pk[i])*Dk[i]*pow(Eo,1+Pk[i])/(1+Pk[i]))
    dEdps = -(M/J)*[1/f_V] # keV.g-1.cm2
    # calculate the deceleration
    
    ##############################################################################################################
    # for the mac database
    # Ionization cross section
    # calculating 1/S
    Uo = Eo/Ec[excited_element]
    
    Vo = V #### IS THIS TRUE???? ####
    m_k=0.86 + 0.12 * math.exp(-pow((Z[Z['element'] == excited_element].atomic_number.values)/5, 2))
    m_values= {'K': m_k,
        'L': 0.82,
        'M': 0.78}
    m = m_values[excited_line]
    T = [1+Pk[0]-m, 1+Pk[1]-m, 1+Pk[2]-m]
    inverse_S = 0
    for i in range(3):
        inverse_S += (Uo/(Vo*M)) * Dk[i]* pow(Vo/Uo, Pk[i]) * (T[i]*pow(Uo, T[i])*math.log(Uo)-pow(Uo,T[i])+1)/pow(T[i],2)
    print("1/S: ", inverse_S)
    ##############################################################################################################
    # Backscatter factor R
    # After calculating R/S, representing the primary intensity, and Q^A_l(Eo), one determines the area F of the distribution phi(pz):
    # F = (R/S)Q

    ######################## Appendix I
    # Z_b_bar
    Z_b_bar = 0
    for element in elements:
        z_element = Z[Z['element'] == element].atomic_number.values
        Z_b_bar += C[z_element-1] * pow(z_element,1/2)
    Z_b_bar = pow(Z_b_bar, 2)
    # eta bar
    eta_bar = 1.75*pow(10,-3)*Z_b_bar + 0.37*(1-math.exp(-0.015*pow(Z_b_bar, 1.3)))
    # W_bar
    W_bar = 0.595 * eta_bar/3.7 + pow(eta_bar,4.55)
    # J(Uo)
    J_Uo = 1+Uo*(math.log(Uo) - 1)
    # G(Uo)
    q = (2 * W_bar - 1) / (1 - W_bar)
    G_Uo = (Uo-1-(1-1/pow(Uo, 1+q))/(1+q))/((2+q)*J_Uo)
    R = 1 - eta_bar * W_bar*(1-G_Uo)
    print("Backscatter factor R:", R)
    # F
    cross_section = math.log(Uo)/(pow(Uo, m)*Ec[excited_element]*Ec[excited_element])
    print('CROSS SECTION: ', cross_section)
    F = (R*inverse_S) / cross_section # it was multiplication but corrected by benoit to be division 
    print('Uo:', Uo)
    print('R :', R)
    print('1/S: ', inverse_S)
    print('Q: ',cross_section)
    print('F:', F)
    ##############################################################################################################
    ######## phi(rhoz)
    r = 2 - 2.3 * eta_bar
    # Phio
    phi0 = 1 + 3.3 * (1-1/pow(Uo,r))* pow(eta_bar, 1.2)
    # Z_bar_n
    lnZ_bar_n = 0
    for element in elements:
        z_element = Z[Z['element'] == element].atomic_number.values
        lnZ_bar_n += C[z_element - 1] * math.log(z_element)
    Z_bar_n = math.exp(lnZ_bar_n)
    # X
    X = pd.DataFrame()
    X_tot = 0
    for idx, value in enumerate(elements):
        # calculating X
        new_row = pd.DataFrame({'element': [value], 'X': mac[(mac['zAbs'] == value) & (mac['zMes'] == excited_element)][excited_line_mac].values[0] * (1/math.sin(math.radians(take_off_angle)))})
        X = X.append(new_row, ignore_index=True)    
        # calculating X total
        z_element = Z[Z['element'] == element].atomic_number.values
        X_tot = X_tot + C[z_element-1] * mac[(mac['zAbs'] == value) & (mac['zMes'] == excited_element)][excited_line_mac].values[0] * (1/math.sin(math.radians(take_off_angle)))

    # Phi: parameters pz, phi0, Z_bar_n, M, J) - output: phi, F1, F2, f_X
    D1 = 6.6*pow(10,-6)
    D2 =1.12*pow(10,-5)*(1.35-0.45*pow(J,2))
    D3 = 2.2*pow(10,-6)/J
    Dk = [D1,D2,D3]
    Pk = [0.78, 0.1, -(0.5-0.25*J)]
    Z_bar = 0
    for element in elements:
        z_element = Z[Z['element'] == element].atomic_number.values
        Z_bar += C[z_element-1] * z_element

    # Rx the range of ionization
    # Ro is the total trajectory of the electrons between Eo and El
    # Q is a function of overvolatge and atomic number
    b = 40/Z_bar
    Qo = 1 - 0.535 * math.exp(-pow(21/Z_bar_n, 1/2)) - 2.5*pow(10,-4)*pow(Z_bar_n/20, 3.5) #page 60 why 1/2 instead of 1.2!!!!!!!!!!!!!!!!!!!!!!!!!
    Q = Qo + (1-Qo)*math.exp(-(Uo-1)/b)
    # D
    h = pow(Z_bar, 0.45)
    D = 1+1/pow(Uo,h)
    # Ro
    Ro = 0
    for i in range(3):
        Ro += (1/M) * pow(J, 1-Pk[i]) * Dk[i] * (pow(Eo,1+Pk[i])-pow(Ec[excited_element], 1+Pk[i]))/(1+Pk[i])

    # Rx
    Rx = Q * D * Ro

    # Rm depth of the maximum of the distribution
    print("Uo-1", Uo-1)
    #if (Uo - 1) > 1:
    G1 = 0.11 * 0.41 * math.exp(-pow(Z_bar/12.75, 0.75))
    G2 = 1 - math.exp(-pow((Uo - 1), 0.35)/1.19)
    G3 = 1 - math.exp(-(Uo-0.5)*pow(Z_bar,0.4)/4)
    Rm = G1 * G2 * G3 * Rx
    # else:
    #     Rm = 0
    #     print('WARNING: ELEMENT NOT EXCITED')
    # d
    d = (Rx-Rm)*((F-(phi0*Rx)/3) * ((Rx-Rm)*F - phi0*Rx*(Rm+Rx/3)))
     # Rc
    Rc = (3/2)
    Rc *= (((F - phi0*Rx/3)/phi0) - (pow(d, 1/2))/(phi0*(Rx-Rm)))

    A1 = phi0 /(Rm*(Rc - Rx * ((Rc/Rm) - 1)))
    B1 = phi0 - A1 * Rm*Rm
    A2 = A1 * (Rc - Rm)/(Rc - Rx)
    # DO we need to calculate this phi?
    # pz = np.linspace(0,1,100)
    # if pz >=0 or pz <= Rc:
    #     phi = A1 * (pz - Rm) * (pz - Rm) + B1
    # if pz > Rc or pz <= Rx:
    #     phi = A2 * (pz - Rx) * (pz - Rx)

    # USING X TOT IS THAT CORRECT?????
    X = X_tot
    # Absorption factor correction
    # F1 is intensity after absorption from 0 to Rc
    # F2 is intensity after absorption from Rc to Rx
    print('##################################################')
    print('Rc: ', Rc)
    print('Rx: ', Rx)
    print('Rm: ', Rm)
    F1 = (A1/X)*(((Rc-Rm) * (Rx-Rc-2/X)-2/(X*X))*math.exp(-X*Rc) - (Rc-Rm)*Rx + Rm*(Rc-2/X) + 2/(X*X))
    # Benoit equations
    Int1atRc = math.exp(-X*Rc)*(-B1*(X*X) -  A1*(2 + 2 *X*(Rc - Rm) + (X*X)* (Rc - Rm)*(Rc - Rm)))/(pow(X,3))
    Int1at0 = (-B1*(X*X) -  A1*(2 - 2 * X * Rm + (X*X)* (Rm*Rm)))/(pow(X,3))
    Int1 = Int1atRc - Int1at0

    F2 = (A2/X)
    F2 *= (((Rx-Rc) * (Rx-Rc-2/X)-2/(X*X))*math.exp(-X*Rc) - (2/(X*X))*math.exp(-X*Rx))
    # Benoit eq
    Int2atRx = -( 2 * A2 * math.exp(-X*Rx) )/(pow(X,3)) #( 2 * A2 .* exp(-Chi.*Rc) )./(Chi.^3);
    Int2atRc = (A2*math.exp(-X*Rc)*(-2-2*X*(Rc - Rx)-(X*X)*(Rc-Rx)*(Rc - Rx)) )/(pow(X,3))
    Int2 = Int2atRx - Int2atRc
    f_X = (F1+F2)/F
    ##############################################################################################################
    print("[+] dE/dps: ", dEdps)
    print("[+] V: ", V)
    print("[+] J: ", J)
    print("[+] M: ", M)
    print("[+] Ro [g/cm2]: ", Ro)
    print('Excited element: ', excited_element)
    print('Ec: ', Ec[excited_element])
    print('Z_bar_n: ', Z_bar_n)
    print('Phi0: ', phi0[0])
    print('M: ', M[0])
    print('F1: ', F1[0])
    print('F1 Benoit', Int1)
    print('F2: ', F2[0])
    print('F2 Benoit', Int2)
    print('f_X: ', f_X[0])

    z_element = Z[Z['element'] == excited_element].atomic_number.values
    intensity = C[z_element-1] * (Int1 + Int2) # C is the concentration of the measured element
    print('Intensity: ', intensity[0])
    print('\n')
    return intensity
# Defining another function that calculate the intensity of all elements in the sample and return a numpy intensity matrix
### SHOULD MAKE THIS RUN FOR ALL ELEMENTS
elements = ['Fe','Ni']
excited_line_list = ['K','K']
excited_line_mac_list = ['Ka', 'Ka']
C = {'Fe':0.1, 'Ni':0.9}
Ec = {'Fe': 7.11, 'Ni': 8.332} ### Check if you have to add all the lines and what to put for H
take_off_angle = 40
Eo = 15 # keV

def CalcI_sample(elements, excited_line_list, excited_line_mac_list, C, Ec):
    elements_df = zaf.get_element(elements)
    C_numpy = np.zeros((92,1))
    for idx, element in enumerate(elements):
        excited_element = element
        excited_line = excited_line_list[idx]
        excited_line_mac = excited_line_mac_list[idx]
        z_element = Z[Z['element'] == element].atomic_number.values
        C_numpy[z_element-1] = CalcI(C, Eo, mac, excited_element, excited_line_mac, excited_line)
    return C_numpy

print(CalcI_sample(elements, excited_line_list, excited_line_mac_list, C, Ec))

1/S:  [[3.67882138e-06]]
Backscatter factor R: [[0.99641447]]
CROSS SECTION:  0.007771196380630213
Uo: 2.109704641350211
R : [[0.99641447]]
1/S:  [[3.67882138e-06]]
Q:  0.007771196380630213
F: [[0.00047169]]
Uo-1 1.109704641350211
##################################################
Rc:  [[2.05540322e-06]]
Rx:  [[0.00059744]]
Rm:  [[2.03656701e-06]]
[+] dE/dps:  [[-194.34381608]]
[+] V:  50.275346541764314
[+] J:  0.2983569688085457
[+] M:  [[0.47590724]]
[+] Ro [g/cm2]:  [[0.00065627]]
Excited element:  Fe
Ec:  7.11
Z_bar_n:  27.793264660963402
Phi0:  [1.48433187]
M:  [0.47590724]
F1:  [4.24893442e-06]
F1 Benoit [[4.24892642e-06]]
F2:  [-2.27594481]
F2 Benoit [[0.00045187]]
f_X:  [-4825.03059938]
Intensity:  [4.56117779e-05]


1/S:  [[1.97911648e-06]]
Backscatter factor R: [[0.99709414]]
CROSS SECTION:  0.005107934269739443
Uo: 1.8002880460873738
R : [[0.99709414]]
1/S:  [[1.97911648e-06]]
Q:  0.005107934269739443
F: [[0.00038633]]
Uo-1 0.8002880460873738
###############################

C:\Users\khalil\AppData\Local\Temp\ipykernel_18900\3159604347.py:116: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X = X.append(new_row, ignore_index=True)


In [21]:
## regression algorithm
Isp = np.zeros((92,1)) # (measured) intensity in specimen
Istd = np.zeros((92,1)) # (measured) intensity in standard

# Calculating IntPAPstd in each standard
nStd = 1
IntPAPstd = np.zeros((92, nStd))
for i in range(nStd):
    IntPAPstd[:,i] = CalcI(Std_AtWtPc(i,:)/100, E0, MACkalpha)

IntPAPstdRshp = IntPAPstd # IntPAPstdRshp = NaN .* Cstd;
KRatiosMsrd = (Isp/Istd)
KRatiosMsrdNorm = KRatiosMsrd
KRatiosMsrdNorm = KRatiosMsrdNorm # to normalize => /repmat(sum(KRatiosMsrdNorm,2),1,92) # normalize

Csp_i = (Isp/Istd) * Cstd # Castaing's approximation for first guess of concentrations
# Csp_inorm = zeros(size(Csp_i)); Csp_inorm(Csp_i==Csp_i) = Csp_i(Csp_i==Csp_i);
# Csp_inorm = Csp_inorm./repmat(sum(Csp_inorm,2),1,92);% normalised
# Csp_i = Csp_inorm

ntests = 1
j = 1
for  i in range(ntests)
    # mdl refer to model
    IntPAPsp_t[:,i] = CalcI(Csp_t, E0, MACkalpha) # Csp_t 
    IntRatio = IntPAPsp_t[:,1]/IntPAPstdRshp[:,j]
    sumKmdl = sum(IntRatio(IntRatio==IntRatio))
    sumKmsrd = sum(KRatiosMsrd) # NEED TO BE MODIFIED
    IntRatioNorm = IntRatio/sumKmdl;

    KRatiosMdlNorm[:,i] = IntRatioNorm #.* Csp_t ./ Cstd(j,:);

    rat =  sumKmsrd/sumKmdl

    Csp_t = Csp_t * KRatiosMsrd[:,j] #./ IntRatio;%

    KRatiosMdl[:,i] = IntRatio * rat #.* Csp_t ./ Cstd(j,:);

    Csps[:,i+1] =  Csp_t # KRatiosMsrd(j,:) .* IntPAPstdRshp(j,:) ./ (rat*IntPAPsp_t(i,:)./Csp_t);

SyntaxError: invalid syntax (3764980469.py, line 9)